# 大模型简介
- 将互联网上的资料压缩在 LLM 中形成概率权重, 然后根据输入(prompt + output)预测下一个单词
- output[n] = f(prompt + output[:n-1])
- 入门文章
  - https://jalammar.github.io/illustrated-gpt2/

# 大模型基本参数
- 基本对话
- top_logprobs
- context size
- logit_bias presence_penalty frequency_penalty 
- structure output


In [1]:
import openai
import os
from dotenv import load_dotenv

load_dotenv(override=True)
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"), base_url=os.getenv("OPENAI_BASE_URL"))
model = os.getenv("OPENAI_MODEL")
# print(client.base_url)
print(model)

deepseek-v3-0324


In [32]:
# non stream
response = client.chat.completions.create(
    model=model,
    messages=[
        {'role': 'user', 'content': 'Count to 10, with a comma between each number and no newlines. E.g., 1, 2, 3, ...'}
    ],
    temperature=0,
    # stream=True,
)

response.choices[0].message.content

'1, 2, 3, 4, 5, 6, 7, 8, 9, 10'

In [30]:
response = client.chat.completions.create(
    model=model,
    messages=[
        {'role': 'user', 'content': 'q: what is the capital of fracnce?\n a:'}
    ],
    temperature=0,
    logprobs=True,
    top_logprobs=2
)
response.choices[0].message.content

'The capital of France is Paris.'

In [31]:
response.to_dict()

{'id': 'chatcmpl-19SN4qBjCbt6cVqJrPR4Pp54RDm354l0',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': {'content': [{'token': 'The',
      'bytes': [84, 104, 101],
      'logprob': -6.630610641877865e-06,
      'top_logprobs': [{'token': 'The',
        'bytes': [84, 104, 101],
        'logprob': -6.630610641877865e-06},
       {'token': 'a', 'bytes': [97], 'logprob': -12.250006675720215}]},
     {'token': ' capital',
      'bytes': [32, 99, 97, 112, 105, 116, 97, 108],
      'logprob': 0.0,
      'top_logprobs': [{'token': ' capital',
        'bytes': [32, 99, 97, 112, 105, 116, 97, 108],
        'logprob': 0.0},
       {'token': 'capital',
        'bytes': [99, 97, 112, 105, 116, 97, 108],
        'logprob': -19.75}]},
     {'token': ' of',
      'bytes': [32, 111, 102],
      'logprob': 0.0,
      'top_logprobs': [{'token': ' of',
        'bytes': [32, 111, 102],
        'logprob': 0.0},
       {'token': ' city',
        'bytes': [32, 99, 105, 116, 121],
        'lo

In [32]:
from IPython.display import display, HTML
import numpy as np

top_two_logprobs = response.choices[0].logprobs.content[0].top_logprobs
html_content = ""
for i, logprob in enumerate(top_two_logprobs, start=1):
    html_content += (
        f"<span style='color: cyan'>Output token {i}:</span> {logprob.token}, "
        f"<span style='color: darkorange'>logprobs:</span> {logprob.logprob}, "
        f"<span style='color: magenta'>linear probability:</span> {np.round(np.exp(logprob.logprob)*100,2)}%<br>"
    )
display(HTML(html_content))
print("\n")

# context size
- 产生的 token 越多, 模型运行过程中要的显存也就越大.
- tokenizer_config.json 中 model_max_length (context size)
    - context size = input (prompt) + output

In [13]:
content = '200个字描述一座山'

resp1 = client.chat.completions.create(
    model=model,
    messages=[
        {'role': 'user', 'content': content}
    ],
)

resp2 = client.chat.completions.create(
    model=model,
    messages=[
        {'role': 'user', 'content': content}
    ],
    presence_penalty=2,
    frequency_penalty=2
)

print(resp1.choices[0].message.content)
print("\n")
print(resp2.choices[0].message.content)

这座山巍峨耸立，宛如大地的守护者，峰顶常年被白雪覆盖，宛如一顶银色的 crown。山体由坚硬的岩石构成，带有复杂的纹理和色彩，映射出岁月的痕迹。清晨时分，阳光透过薄雾，给山脊镶上金边，显得格外神秘。山脚下，溪水潺潺，潺潺而流，鱼儿在水中嬉戏，周围的树木郁郁葱葱，仿佛在向游客诉说着这座山的古老传说。

每到秋天，山上的树叶变幻出绚丽的色彩，红、黄、橙相互交织，犹如一幅美丽的画卷。登顶之路蜿蜒曲折，需经历艰难的攀登，但当你站在山顶，俯瞰四周，壮丽的景色顿时令你心旷神怡。远方的群山连绵起伏，仿佛在向你讲述自然的伟大与神秘。这座山，不仅是大自然的杰作，更是人们心灵深处的向往和归宿。


这座山巍峨雄伟，屹立在天地之间。山体披着厚厚的绿毯，苍翠欲滴的森林覆盖其间，让人仿佛置身于一片天然氧吧。阳光透过树梢洒下斑驳陆离的光影，鸟儿鸣唱声此起彼伏，为宁静的大自然增添了几分生机。

随着高度逐渐增加，可以看到层层梯田和溪流蜿蜒而下，如丝带一般环绕四周。在山顶俯瞰，更是壮观无比：群峰耸立、云海翻腾，让人心旷神怡。有时薄雾缭绕，就像给这座独特的地方蒙上了一层神秘面纱。

这里不仅是登高望远之地，也是许多珍稀动物与植物的天堂，是探险爱好者和摄影师们梦寐以求之所。在这样的环境中，人们能感受到大自然赋予的一种震撼力量，也让自己的内心得到洗礼与升华。这是一处让人与自然融为一体、沉浸于纯粹美好的精神乐园。


# tokenizer & logit_bias 

In [3]:
import tiktoken
tokenizer = tiktoken.encoding_for_model(model)

In [4]:
print(tokenizer.encode(' Paris'))
print(tokenizer.decode([12650]))

[12650]
 Paris


In [5]:
resp = client.chat.completions.create(
    model=model,
    messages=[
        {'role': 'user', 'content': 'q: what is the capital of fracnce?\n a:'}
    ],
    logprobs=True,
    top_logprobs=2,
    temperature=0,
    logit_bias={12650:-100, 72782:-100}
)

In [23]:
resp.choices[0].message.content

'The capital of France is باريس (in French: "Pariser").'

In [24]:
resp.to_dict()

{'id': 'chatcmpl-r93AVey3euLTcvr7UAr4PP54RDM72tu0',
 'choices': [{'finish_reason': 'stop',
   'index': 0,
   'logprobs': {'content': [{'token': 'The',
      'bytes': [84, 104, 101],
      'logprob': -3.8889261304575484e-06,
      'top_logprobs': [{'token': 'The',
        'bytes': [84, 104, 101],
        'logprob': -3.8889261304575484e-06},
       {'token': 'a', 'bytes': [97], 'logprob': -12.875003814697266}]},
     {'token': ' capital',
      'bytes': [32, 99, 97, 112, 105, 116, 97, 108],
      'logprob': 0.0,
      'top_logprobs': [{'token': ' capital',
        'bytes': [32, 99, 97, 112, 105, 116, 97, 108],
        'logprob': 0.0},
       {'token': 'capital',
        'bytes': [99, 97, 112, 105, 116, 97, 108],
        'logprob': -19.75}]},
     {'token': ' of',
      'bytes': [32, 111, 102],
      'logprob': 0.0,
      'top_logprobs': [{'token': ' of',
        'bytes': [32, 111, 102],
        'logprob': 0.0},
       {'token': ' city',
        'bytes': [32, 99, 105, 116, 121],
        '

In [1]:
from transformers import AutoTokenizer
tokenizer_transformer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")

In [4]:
print(tokenizer_transformer.encode(' Paris'))
print(tokenizer_transformer.decode([12095]))

[49434, 239]
[12095]
 Paris


# Structured Outputs

In [48]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model=model,
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

print(completion.choices[0].message.content)
event = completion.choices[0].message.parsed

{"date":"2023-10-06","name":"Science Fair","participants":["Alice","Bob"]}


In [46]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "Who won the world series in 2020? Please respond in the format {winner: ...}"}
    ],
    response_format={"type": "json_object"}
)
print(response.choices[0].message.content)

{'id': 'chatcmpl-vzYwDXJ2qDLscvPv1RJ4pp54rDM07230', 'choices': [{'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '{ "winner": "Los Angeles Dodgers" }', 'role': 'assistant'}}], 'created': 1744040175, 'model': 'gpt-3.5-turbo-0125', 'object': 'chat.completion', 'usage': {'completion_tokens': 11, 'prompt_tokens': 41, 'total_tokens': 52}}
{'content': '{ "winner": "Los Angeles Dodgers" }', 'role': 'assistant'}


大模型并非全能, 而是在海量数据集下, 学会了"猜"出最可能的下一个词或句。数据集中某个领域知识越多， 它就猜的越准。

![ai](../img/ai_and_huan.PNG)

- 要深刻理解哪些是 AI 知道的, 哪些 AI 不知道的
    - AI 知道: 简单讲
        - 请你用鲁迅的文风帮我写一篇关于体育圈饭圈文化的文章
        - 请用通俗易懂的语言帮我解释下祛魅这个词
    - 人知道, AI 不知道的事情: 用投喂的方式
        - B 站的辉耀计划视频风格文案 --> 举例子
        - B 站何同学的微博发言 

- 优化大模型手段
    - 提示词工程
    - 微调
    - RAG

# Prompt engineering
- 结构化提示词
    - 明确任务
    - 补充背景
    - 设定角色
    - 给例子
    - 规定格式
- deepseek R1 推理模型技巧
    - 比如: 我要 xx, 要给 xx 用, 希望达到 xx 效果, 但担心 xxx 问题
    - 加入 说人话 提示词
    - 多肯定，反复迭代追问多试几次
- 升级提问, 逐级降维(不知道怎么提问, 直接询问 AI 如何提问)
    - 我刚参加完一个会议，需要整理一份会议纪要。我不确定该从哪些方面入手，你能给我一些建议吗？
    - 如何制定一份健身计划
- reference
    - 视频
        - https://www.bilibili.com/video/BV1iuXkYHE2B
        - https://www.bilibili.com/video/BV12JtpeiEvq
    - 文档
        - https://baoyu.io/blog/google-prompt-engineering-whitepaper
        - https://dannyzheng.me/


In [13]:
from concurrent.futures import ThreadPoolExecutor

content1 = "在版本号命名规则中, 13.11 和 13.8 那个大?"
content2 = "在数学中, 13.11 和 13.8 那个大?"

def get_response(model, content):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": content}],
    )
    return response

with ThreadPoolExecutor() as executor:
    custom_model = "deepseek-v3"
    # 并行发起两个请求
    future1 = executor.submit(get_response, model,  content1)
    future2 = executor.submit(get_response, model, content2)

    response1 = future1.result()
    response2 = future2.result()
    print(response1.choices[0].message.content)
    print("=" * 100)
    print(response2.choices[0].message.content)

在标准的版本号命名规则（例如语义化版本 SemVer）中，版本号是按部分从左到右进行比较的。

比较 13.11 和 13.8：

1.  **比较第一部分（主版本号 Major）：** 都是 13，相等。
2.  **比较第二部分（次版本号 Minor）：** 11 和 8。
3.  **判断大小：** 因为 11 大于 8。

所以，**13.11** 比 13.8 大（或者说是更新的版本）。
在数学中比较 13.11 和 13.8 的大小：

1.  **比较整数部分：** 两个数的整数部分都是 13，相同。
2.  **比较小数部分（从左到右）：**
    *   比较小数点后第一位（十分位）：13.11 的是 1，13.8 的是 8。
    *   因为 8 大于 1。

所以，**13.8** 大于 13.11。

简单来说，你可以把 13.8 看作 13.80。这样比较 13.11 和 13.80 就更直观了，因为 80 大于 11，所以 13.80 (即 13.8) 大于 13.11。


In [ ]:
content1 = """<example> 以前打网约车，司机师傅跟我说打个好评，我都会说好好好，但是下车后也没想起来打。
其实这样挺不好的。现在司机师傅跟我说打个好评，除非服务真的很好到我想打好评的程度，否则我就会直接说，抱歉我不想打，然后下车。
作为一个有讨好倾向的人，这是我锻炼真诚和勇气的方式。</examle>
仿照上面 example 标签中语句, 写一段b站看视频一键三连的话术"""

response1 = client.chat.completions.create(
    model=model,
    messages=[
        {'role': 'user', 'content': content1}
    ],
)

print(response1.choices[0].message.content)

以前在B站看视频，主播常常提醒要一键三连，我总是答应着“好好好”，但其实下了视频就忘记了。 这样其实不太好。现在如果主播说要一键三连，除非视频真的让我觉得很有意思、很有价值，否则我会诚实地告诉他们，抱歉我不打算三连，然后就离开。 作为一个喜欢支持创作者的人，这也是我锻炼诚实和勇气的方式。


In [4]:
content1 = """<example>
为什么需要 A
什么是 A
怎么使用 A
使用 A 时注意的问题
A 的应用领域
A 的优缺点
A 是否重要, 是否需要花精力钻研 (初学者, 专家级别)</example>
请你按照上面 example 标签中思考逻辑, 回答编程语言中的 泛型"""

response1 = client.chat.completions.create(
    model=model,
    messages=[
        {'role': 'user', 'content': content1}
    ],
)

print(response1.choices[0].message.content)


### 为什么需要泛型
泛型使得编程更加灵活和可复用。它允许开发者编写可以处理多种数据类型的代码，从而避免了代码重复和强制类型转换的需求。同时，泛型可以提高类型安全性，降低运行时错误的可能性。

### 什么是泛型
泛型是一种编程语言特性，允许函数、类或接口在定义时不指定具体的数据类型，而是在具体使用时提供数据类型。这样，开发者可以创建可以适用于多种数据类型的代码。

### 怎么使用泛型
在使用泛型时，通常会在函数、类或接口的定义中引入类型参数。例如，在 Java 中，可以定义一个泛型类如下：

```java
public class Box<T> {
    private T item;

    public void setItem(T item) {
        this.item = item;
    }

    public T getItem() {
        return item;
    }
}
```

使用时可以指定具体的数据类型：

```java
Box<Integer> integerBox = new Box<>();
integerBox.setItem(123);
Integer item = integerBox.getItem();
```

### 使用泛型时注意的问题
1. **类型擦除**：在一些语言（例如 Java）中，泛型在运行时会被擦除为原始类型，需要注意类型信息可能无法在运行时获取。
2. **不能使用基本类型**：一些语言不允许直接使用基本数据类型作为泛型参数，需使用对应的包装类。
3. **有界参数**：如果需要限制泛型类型的范围，可以使用有界类型参数，但这可能增加复杂性。

### 泛型的应用领域
泛型广泛应用于数据结构（如集合、列表、树等）、算法（例如排序、搜索）和API设计中。许多现代编程语言（如 Java、C#、C++ 和 Swift）都支持泛型。

### 泛型的优缺点
**优点**：
- 代码复用性高：支持多种数据类型，可以减少代码重复。
- 类型安全性高：减少了因类型不匹配导致的运行时错误。
- 提高可读性：代码更加简洁清晰，易于理解。

**缺点**：
- 学习曲线：初学者可能需要时间理解泛型的概念及其用法。
- 复杂性增加：在一些情况下，泛型代码可能难以理解和调试。
- 